In [7]:
%run ItemModel.ipynb
%run APIService.ipynb

In [8]:
import inspect, re
import pandas as pd
import csv
from pathlib import Path
import shutil

# Class DataCrawler

In [9]:
class DataCrawler(apiServiceDelegate):
    max_page = -1
    page = 1
    start_page = 1
    end_page = 2
    query = "environment concept art"
    source = 'artstation'
    per_page = '75'
    items_json = []
    items_img_json = []
    all_items = [] # List ARTSTATIONItem
#     all_links = []
    itemModel = ARTSTATIONItem()
    noneCategory = True
    APIService = APIService(source, query, per_page)
    i = 0
    
    @property
    def fileName(self):
        return f'ArtStationItem_page{self.page}.tsv'
    
    def __init__(self, source = 'artstation', query = 'environment concept art', per_page = '75', max_page = -1):
        self.source = source
        self.query = query
        self.per_page = per_page
        self.max_page = max_page
        self.APIService = APIService(self.source, self.query, self.per_page)
        self.APIService.delegate = self
        self.savePath = '/Users/khangnguyen/Documents/GitHub/Delegation-Pattern-In-Python-To-Crawl-Data-With-API/application/artstation/img_download'
        pass
    def setPage(self, page):
        self.page = page
        pass
    
    def pipeline_on_page(self, start_page = 1, end_page = 200):
        self.start_page = start_page
        pass
    
    def crawlData(self, start_page = 1, end_page = 200):
        while True:
            try:
                check = False
                if start_page <= self.page and self.page < end_page:
                    self.APIService.setPage(self.page)
                    check = self.APIService.get()
                if check == False: break
                elif self.max_page != -1 and self.page == (self.max_page + 1): break
                
            except Exception as e:
                self.parseData()
                print(f'Error: {e}')
                break
        pass
    
    def crawlImage(self):
        if len(self.all_items) != 0:
            for j in range(len(self.all_items)):
                try:
                    self.APIService.img_hash_id = self.all_items[j].hash_id
                    check = self.APIService.get_img()
                    if check == False: break
                except Exception as e:
#                     self.parseData('img')
                    print(f'Error Crawl Image: {e}')
                    break
                self.i += 1
        else:
            print('len(all_items) == 0')
        pass
    
    def parseData(self):
        self.all_items.clear()
        for item in self.items_json:
            itemModel = ARTSTATIONItem()
            itemModel.hash_id = item['hash_id']
            
            self.all_items.append(itemModel)
            del itemModel
            itemModel = None
        pass
    def parseImage(self):
        print('parseImage ', self.i, ': ', len(self.items_img_json), ' url')
        itemModel = []
#         itemModel.append(['.'])
        for item in self.items_img_json:
# #             print(item['image_url'])
            itemModel.append(item['image_url'])
# #         print('---',itemModel.link)
                
#         self.all_items[self.i].link = ['1']
        self.all_items[self.i].link = itemModel
        del itemModel
        itemModel = None
        pass
    
    def saveFile(self, name = fileName):
        self.fileName = name
        file = open(self.fileName, 'w', encoding='utf-8')
 
        file.write(f'hash_id\turl\n')
 
        
        for item in self.all_items:
                file.write(f'{item.hash_id}\t{item.link}\n')
    
    def downloadImage(self):
        p = Path(self.savePath)
        for item in self.all_items:
            for linkIdx in range(len(item.link)):
                r = requests.get(item.link[linkIdx], stream=True)
                if r.status_code == 200:
                    with open(f"{p}/{item.hash_id}_{linkIdx}.jpg", 'wb') as f:
                        r.raw.decode_content = True
                        shutil.copyfileobj(r.raw, f)
                
        pass
                
    def updateDataWhenSuccess(self, json_pydata):
        print(len(json_pydata['data']))
        print(f'get! {self.page}')
        
        if 'data' in json_pydata:
            self.items_json.extend(json_pydata['data'])
        else:
            self.parseData()
            print('get All Data!')
            return False

        self.page += 1
        time.sleep(3)
        return True
        pass
    def getDataError(self, error):
        self.parseData()
        print(f'Error when request: {error}')
        return False
        pass
    def updateImageWhenSuccess(self, json_pydata):
#         print(f'Update link at item {self.i}')
        self.items_img_json.clear()
        if 'assets' in json_pydata:
            self.items_img_json = json_pydata['assets']
            self.parseImage()
        else:
#             self.parseData('img')
            print('get All Image Link!')
            return False
#         print(f' Done Update link at item {self.i}')
        time.sleep(0.3)
        return True
        pass
    def getImageError(self, error):
        print(f'Delegate: getImageError: {error}')
#         self.parseData('img')
        return False
        pass    
    pass

# Crawling Data

In [22]:
crawler = DataCrawler(per_page = 3,max_page=1)

In [23]:
crawler.crawlData()


3
get! 1


In [24]:
crawler.parseData()

In [34]:
print(f'{crawler.all_items[0].hash_id}')

4boNZ4


TODO: This is the code from November 2021, now October 2022, for some reason, looks like it crawls all the data available with `per_page = 3, max_page=1`, so:

 - Check the API call
 - If you decide to use this code, make sure to check attribute `all_items` to not have any duplicate value, and limit it, because calling the method `crawlImage()` will take time.

In [35]:
len(crawler.all_items)

9983

In [38]:
for i in crawler.all_items:
    print(i.hash_id)

4boNZ4
KrJKLx
Eg1Aq
DwaWy
xoNEX
ykO0gR
B16OD8
9XZgR
VD5QZ
A9xRn5
ZYWO8
xkxAm
R3eJar
b1ABv
LPAYA
480ow2
dO8NlJ
PbG94
Ko9e4
xJelOY
qg9Q2
q9v5Re
B1ZJQA
q9PR8L
4bylE1
RYz64y
xJ2OQE
dXD1e
68RG4x
e0z1lX
VdmLx4
2zZvg
L2ObRK
eWqXD
5l2v1
A9orae
Qr5nGl
8g8PO
Rwelm
JleeZa
2qqxqa
BmOLbD
8ex1ZG
ky1x6
9BV6q
mqlK88
J9JJQ0
m4Zd
L2OwzK
lnnyJ
ak6Kz
Ooz2Rk
z5gom
VdkWdn
babY5G
12662
g8LL88
6aymDn
eaZPqY
gP8qE
Edqn
6b2lDW
Z5OVzN
lVLrJJ
3q11m
oO8K2k
KW69G
KOBPL9
R3VPKA
roLw6
mDorw8
oA4yNq
RJ2Yr
4X4k0Y
xYv29O
NPPJd
b5ZVOg
XKe8D
d0n2yQ
JezKZ0
wPx59
Jlk8vR
qAO4m2
PoxKQ4
G80AoW
489WOq
486Waq
qAKyxy
bK36yG
JeJVOa
28Q4vA
3orVAJ
lkXqV
k4GgmK
XnrLAl
dOgJ5e
yJe2X9
Y0o2b
QnXKAx
Q1ZDr
aY4P1q
kDKoby
lVdA45
Vge29N
3gaXv
q53Vy
Ya8LJ3
r95dYe
VdZzN
KrNZ4W
68ZlqO
AJwYq
WmxkyD
DA1yRG
g8d5rZ
2neOK
5B29e1
L15G5
OmeYXg
B2Ez6
WKq6yy
3qWZom
vDq5E
D5nqAn
xJPQNY
N5Keqq
dOblqA
WKEbAX
wJYD1g
Dx49Je
58XqE8
OmBEQ8
LegRav
zDrXXL
ELXOxK
2qKYLg
3doDyA
YKvBnV
WmoWVQ
RRErO
kl6P5n
Xn1v1L
lV8DJe
14bgkX
KOkOxB
6b0Z2O
AqVEEV
YKO1QX
g2QE6L
XY0Bn

In [32]:
crawler.crawlImage()

parseImage  0 :  14  url
parseImage  1 :  4  url
parseImage  2 :  4  url
parseImage  3 :  1  url
parseImage  4 :  1  url
parseImage  5 :  6  url
parseImage  6 :  5  url
parseImage  7 :  1  url
parseImage  8 :  6  url
parseImage  9 :  1  url
parseImage  10 :  4  url
parseImage  11 :  1  url
parseImage  12 :  10  url
parseImage  13 :  6  url
parseImage  14 :  2  url
parseImage  15 :  4  url
parseImage  16 :  1  url
parseImage  17 :  1  url
parseImage  18 :  2  url
parseImage  19 :  4  url
parseImage  20 :  4  url
parseImage  21 :  4  url
parseImage  22 :  9  url
parseImage  23 :  2  url
parseImage  24 :  3  url
parseImage  25 :  4  url
parseImage  26 :  1  url
parseImage  27 :  2  url
parseImage  28 :  1  url
parseImage  29 :  7  url
parseImage  30 :  5  url
parseImage  31 :  7  url
parseImage  32 :  3  url
parseImage  33 :  2  url
parseImage  34 :  2  url
parseImage  35 :  2  url
parseImage  36 :  1  url
parseImage  37 :  2  url
parseImage  38 :  5  url
parseImage  39 :  8  url
parseIma

KeyboardInterrupt: 

In [36]:
crawler.downloadImage()

KeyboardInterrupt: 

In [ ]:
# crawler.items_img_json

In [ ]:
crawler.all_items[9].link

In [ ]:
# Check last request
# print(json.dumps(crawler.APIService.json_pydata['data'], indent = 3, sort_keys=True))

In [ ]:
# Check feature value
crawler.all_items[0].hash_id

In [ ]:
# Test ParseData
print(crawler.all_items[0].hash_id)
print(crawler.all_items[9].hash_id)
# print(crawler.all_items[10].hash_id)
# print(crawler.all_items[19].hash_id)

In [9]:
crawler.saveFile('art.csv')

In [10]:
# Test saveFile
items_df = pd.read_csv('art.csv', sep='\t')
pd.set_option('max_colwidth', None)

items_df



,hash_id,url
0,4boNZ4,"['https://cdna.artstation.com/p/assets/covers/images/022/118/768/large/christian-dimitrov-02.jpg?1574182578', 'https://cdna.artstation.com/p/assets/images/images/028/839/734/large/christian-dimitrov-01-legendary-2500.jpg?1595675385', 'https://cdna.artstation.com/p/assets/images/images/028/839/740/large/christian-dimitrov-02-legendary-2500.jpg?1595675393', 'https://cdna.artstation.com/p/assets/images/images/028/839/744/large/christian-dimitrov-03-legendary-2500.jpg?1595675402', 'https://cdnb.artstation.com/p/assets/images/images/028/839/749/large/christian-dimitrov-04-legendary-2500.jpg?1595675410', 'https://cdnb.artstation.com/p/assets/images/images/028/839/785/large/christian-dimitrov-05-legendary-2500.jpg?1595675484', 'https://cdna.artstation.com/p/assets/images/images/028/839/788/large/christian-dimitrov-06-legendary-2500.jpg?1595675490', 'https://cdnb.artstation.com/p/assets/images/images/028/839/789/large/christian-dimitrov-07-legendary-2500.jpg?1595675496', 'https://cdnb.artstation.com/p/assets/images/images/028/839/791/large/christian-dimitrov-08-legendary-2500.jpg?1595675502', 'https://cdnb.artstation.com/p/assets/images/images/036/780/985/large/christian-dimitrov-09-legendary-2500.jpg?1618588862', 'https://cdna.artstation.com/p/assets/images/images/028/839/812/large/christian-dimitrov-10-legendary-2500.jpg?1595675632', 'https://cdnb.artstation.com/p/assets/images/images/028/839/815/large/christian-dimitrov-11-legendary-2500.jpg?1595675637', 'https://cdna.artstation.com/p/assets/images/images/028/839/818/large/christian-dimitrov-12-legendary-2500.jpg?1595675644', 'https://cdna.artstation.com/p/assets/images/images/036/780/988/large/christian-dimitrov-13-legendary-2500.jpg?1618588874']"
1,KrJKLx,"['https://cdna.artstation.com/p/assets/images/images/033/530/544/large/sergey-samsonov-1-concept-environment-cargarbage-visha-published.jpg?1609869861', 'https://cdnb.artstation.com/p/assets/images/images/033/529/927/large/sergey-samsonov-2-concept-environment-cargarbage-office-sails-ublished.jpg?1609868704', 'https://cdnb.artstation.com/p/assets/images/images/033/529/931/large/sergey-samsonov-3-concept-environment-cargarbage-old-elevator-published.jpg?1609868710', 'https://cdnb.artstation.com/p/assets/images/images/033/529/937/large/sergey-samsonov-4-concept-environment-cargarbage-published.jpg?1609868718']"
2,DwaWy,['https://cdna.artstation.com/p/assets/images/images/008/897/794/large/branislav-perkovic-13.jpg?1515977576']
3,xoNEX,['https://cdna.artstation.com/p/assets/images/images/004/247/572/large/michel-donze-forestsconcept-upper.jpg?1481706939']
4,Eg1Aq,"['https://cdnb.artstation.com/p/assets/images/images/004/900/545/large/francis-tneh-pitch-pack-cyberpunk-downtown-final.jpg?1487085945', 'https://cdna.artstation.com/p/assets/images/images/004/900/544/large/francis-tneh-ds-env-01.jpg?1487085876', 'https://cdnb.artstation.com/p/assets/images/images/004/900/547/large/francis-tneh-pitch-pack-darkelf-palace-final.jpg?1487085884', 'https://cdna.artstation.com/p/assets/images/images/004/900/548/large/francis-tneh-pitch-pack-elf-palace-final.jpg?1487085889']"
...,...,...
9970,YlR53,[]
9971,1DAw2,[]
9972,oWZXk,[]
9973,Govnz,[]


In [ ]:
# Migrate Data
# Run this if you run the crawler before it've been updated to auto wirte the sell_time feature.

# items_df.insert(4, 'sell_time', items_df['item_be_got_time'] - items_df['item_public_time'])
# items_df.to_csv('ShoppeItem_thoiTrangNam_aoKhoacAoVest_0-949.tsv', sep = '\t', index=False)

In [ ]:
del items_df